# Running population analysis with `GWPopulation`

First, before running this you should install `gwpopulation` and `cupy` if you have access to a Nvidia GPU.

```
pip install gwpopulation
pip install cupy
```

## `GWPopulation`

`gwpopulation` is a library for performing GPU-accelerated population inference with a focus on gravitational wave populations.

If `cupy` is installed the GPU will be used, if not the performance will suffer but it will fall back to `numpy`.

Builds upon [`bilby`](git.ligo.org/lscsoft/bilby) ([arXiv:1811.02042](https://arxiv.org/abs/1811.02042)) to provide simple, modular, user-friendly, population inference.

Currently implemented models include:
- One and two component mass distributions in primary mass and mass ratio, e.g., Talbot & Thrane (2018) ([arXiv:1801:02699](https://arxiv.org/abs/1801.02699)), Fishbach & Holz (2018) ([arXiv:1709.08584](https://arxiv.org/abs/1709.08584)).
- The same mass distributions but independent but identically distributed primary and secondary.
- Half-Gaussian + isotropic spin tilt distribution from Talbot & Thrane (2017) ([arXiv:1704.08370](https://arxiv.org/abs/1704.08370)).
- Beta spin magnitude distribution from Wysocki+ (2018) ([arXiv:1805:06442](https://arxiv.org/abs/1805.06442)).
- Each of these are also available with independent but identically distributed spins.
- Redshift evolution model as in Fishbach+ (2018) ([arXiv:1805.10270](https://arxiv.org/abs/1805.10270)).
- More to come and any contributions welcome...

For more information see the [git repository](https://github.com/ColmTalbot/gwpopulation), [documentation](https://colmtalbot.github.io/gwpopulation/), or [paper](https://dcc.ligo.org/LIGO-P1900101).

If you want to try out a GPU-accelerated version you can open [this notebook](https://github.com/ColmTalbot/gwpopulation/blob/master/examples/GWTC1.ipynb) in [colab](colab.research.google.com).
It runs through a similar demonstration without accessing any proprietary data.

In [ ]:
%pylab inline

import h5py

import pandas as pd
from scipy.interpolate import interp1d, RegularGridInterpolator
import deepdish as dd
from astropy import cosmology, units

import bilby as bb
from bilby.core.prior import LogUniform, PriorDict, Uniform
from bilby.hyper.model import Model
import gwpopulation as gwpop
xp = gwpop.cupy_utils.xp

# Load posteriors

We're using the posteriors from the GWTC-1 data release.

We need to change the names of the parameters to make them work with the code.

In [ ]:
parameter_translator = dict(
    mass_1_det='m1_detector_frame_Msun',
    mass_2_det='m2_detector_frame_Msun',
    luminosity_distance='luminosity_distance_Mpc',
    a_1='spin1',
    a_2='spin2',
    cos_tilt_1='costilt1',
    cos_tilt_2='costilt2')

posteriors = list()
priors = list()

events = ['150914', '151012', '151226', '170104', '170608',
          '170729', '170809', '170814', '170818', '170823']
for event in events:

    _posterior = pd.DataFrame()
    _prior = pd.DataFrame()
    with h5py.File('/home/colm.talbot/event_posteriors/GWTC1/GWTC-1_sample_release/GW{}_GWTC-1.hdf5'.format(event)) as ff:
        for my_key, gwtc_key in parameter_translator.items():
            _posterior[my_key] = ff['IMRPhenomPv2_posterior'][gwtc_key]
            _prior[my_key] = ff['prior'][gwtc_key]
    posteriors.append(_posterior)
    priors.append(_prior)

In [ ]:
luminosity_distances = np.linspace(1, 10000, 1000)
redshifts = np.array([
    cosmology.z_at_value(cosmology.Planck15.luminosity_distance, dl * units.Mpc)
    for dl in luminosity_distances])
dl_to_z = interp1d(luminosity_distances, redshifts)

luminosity_prior = luminosity_distances ** 2

dz_ddl = np.gradient(redshifts, luminosity_distances)

redshift_prior = interp1d(redshifts, luminosity_prior / dz_ddl)

## Add some weights to posterior

Make sure the posterior `DataFrames` contain the appropriate quantities.

We could include a `prior` column, this is the prior used in the initial sampling stage.
This is used to weight the samples in the likelihood.

In [ ]:
for posterior in posteriors:
    posterior['redshift'] = dl_to_z(posterior['luminosity_distance'])
    posterior['mass_1'] = posterior['mass_1_det'] / (1 + posterior['redshift'])
    posterior['mass_2'] = posterior['mass_2_det'] / (1 + posterior['redshift'])
    posterior['mass_ratio'] = posterior['mass_2'] / posterior['mass_1']

# Specify the model

Choose which population models we want to use, here we'll use mass model B from the populations paper, a powerlaw mass distribution with powerlaw mass ratio distribution.

In [ ]:
model = Model([gwpop.models.mass.power_law_primary_mass_ratio])

# Selection effects

In [ ]:
_vt_data = dd.io.load('/home/colm.talbot/O2/population/paper_runs/vt.h5')
vt_data = dict()
vt_data['mass_1'] = _vt_data['m1']
vt_data['mass_ratio'] = _vt_data['q']
vt_data['vt'] = _vt_data['vt_early_high'] * _vt_data['quadratic_calibration']
vt_evaluator = gwpop.vt.GridVT(
    model=gwpop.models.mass.power_law_primary_mass_ratio, data=vt_data)

# Define the likelihood

In this notebook we use a likelihood which analytically marginalises over the local merger rate assuming a uniform-in-log prior.
If you want to estimate the rate use the `RateLikelihood` and add `rate` as a parameter.
This is done further on in the notebook.

We provide:
- `posteriors`: a list of `pandas` DataFrames
- `hyper_prior`: our population model, as defined above
- `sampling_prior`: this is being depracated
- `selection_function`: anything which evaluates the selection function

We can also provide:
- `max_samples`: the maximum number of samples to use from each posterior, this defaults to the length of the shortest posterior
- `conversion_function`: a function which takes the sampled parameter dictionary and returns a dictionary also containing the parameters required for the population model.

We get a warning telling us `cupy` is not available and so `numpy` is for the likelihood evaluation.
This will go away if you have a GPU and `cupy` installed.

In [ ]:
fast_likelihood = gwpop.hyperpe.HyperparameterLikelihood(
    posteriors=posteriors, hyper_prior=model, selection_function=vt_evaluator)

# Define the prior

This is the standard method to define the prior distribution within `bilby`.

The labels are used in plotting.

Numbers are converted to delta function priors and are not sampled.

There are many other distributions available, see the code/documentation for a full list.

In [ ]:
fast_priors = PriorDict()

fast_priors['alpha'] = Uniform(minimum=-2, maximum=4, latex_label='$\\alpha$')
fast_priors['beta'] = Uniform(minimum=-4, maximum=12, latex_label='$\\beta$')
fast_priors['mmin'] = Uniform(minimum=5, maximum=10, latex_label='$m_{\\min}$')
fast_priors['mmax'] = Uniform(minimum=20, maximum=60, latex_label='$m_{\\max}$')
fast_priors['lam'] = 0
fast_priors['mpp'] = 35
fast_priors['sigpp'] = 1

# Run the sampler

We'll use the sampler `dynesty` and use a small number of live points.

This is painfully slow without using the GPU version.
If you have a GPU it will just work.

On jupyter.ligo.caltech.edu the CPU version ran in ~30 minutes.
On colab.research.google.com this ran in a GPU runtime is ~30 seconds.

Other samplers are available, `cpnest` gave the best results for the O1+O2 data, however it isn't currently compatible with the GPU likelihood.

_Note_:
With this few live points the sampler might throw some warnings.
They'll go away with more sensible settings.

In [ ]:
fast_result = bb.run_sampler(
    likelihood=fast_likelihood, priors=fast_priors, sampler='dynesty', nlive=100,
    label='fast')

In [ ]:
_ = fast_result.plot_corner(save=False)

## Define a new model

### Let's define a new population model for BNS. 

Just as an example we'll use a Gaussian distribution bounded between $[1 M_{\odot}, 2 M_{\odot}]$.

$$p(m_1, m_2) = N \exp \left(- \frac{\left((m_1 - \mu)^2 + (m_2 - \mu)^2\right)}{2 \sigma^2}\right) \quad : \quad 1 \leq m_2 \leq m_1 \leq 2$$

We see that this function takes three arguments:
- `dataset`: this is common to all of the population models in `gwpopulation`, it is a dictionary containing the data to be evaluated, here it is assumed to contain entries for `mass_1` and `mass_2`, the _source-frame_ masses.
- `mu_bns`: the peak of the bns mass distribution.
- `sigma_bns`: the width of the bns mass distribution.

In [ ]:
def truncated_gaussian_primary_secondary_identical(dataset, mu_bns, sigma_bns):
    prob = gwpop.utils.truncnorm(
        dataset['mass_1'], mu=mu_bns, sigma=sigma_bns, low=1, high=2)
    prob *= gwpop.utils.truncnorm(
        dataset['mass_2'], mu=mu_bns, sigma=sigma_bns, low=1, high=2)
    prob *= (dataset['mass_1'] >= dataset['mass_2'])
    prob *= 2
    return prob

## Load GW170817 posterior

This is just the same as above.

In [ ]:
posterior = pd.DataFrame()
prior = pd.DataFrame()
with h5py.File('/home/colm.talbot/event_posteriors/GWTC1/GWTC-1_sample_release/GW170817_GWTC-1.hdf5') as ff:
    for my_key, gwtc_key in parameter_translator.items():
        try:
            posterior[my_key] = ff['IMRPhenomPv2NRT_lowSpin_posterior'][gwtc_key]
            prior[my_key] = ff['IMRPhenomPv2NRT_lowSpin_prior'][gwtc_key]
        except ValueError:
            pass
        
posterior['redshift'] = dl_to_z(posterior['luminosity_distance'])
posterior['mass_1'] = posterior['mass_1_det'] / (1 + posterior['redshift'])
posterior['mass_2'] = posterior['mass_2_det'] / (1 + posterior['redshift'])

## Define the new likelihood

We use the same likelihood as before.

_Note_:
- This time we cast our posterior to a list while creating the likelihood.
- We pass the function rather than a `Model` object as before, `bilby` will turn this into a `Model` for internal use.
- We've removed the selection and conversion functions as they aren't needed here (yes, a selection function is techinically needed).

In [ ]:
bns_likelihood = gwpop.hyperpe.HyperparameterLikelihood(
    posteriors=[posterior], hyper_prior=truncated_gaussian_primary_secondary_identical)

## Define the new prior

Just as before.

In [ ]:
bns_priors = PriorDict()
bns_priors['mu_bns'] = Uniform(minimum=1, maximum=2, latex_label='$\\mu_{bns}$')
bns_priors['sigma_bns'] = LogUniform(minimum=1e-2, maximum=1, latex_label='$\\sigma_{bns}$')

## Run sampler

The sampler is run as before.

We've changed the:
- `label`
- `nlive`, number of live points, this run is faster, so we can use more live points in a quick run

Unsurprisingly, the posterior peaks slightly at the equal mass limit of 170817 but essentially returns the prior.

In [ ]:
bns_result = bb.run_sampler(
    likelihood=bns_likelihood, priors=bns_priors, sampler='dynesty',
    nlive=1000, label='bns')

_ = bns_result.plot_corner(save=False)

## Do it all

#### Let's put together a run with models for the mass, spin and redshift distributions.

This will take longer to run and requires a model for VT including the effect of redshift evolution.

_Note_:
- the redshift model is a class and so is called slightly differently. This is to enable caching of expensive data internally.
- this currently doesn't work with the pip release of `bilby`, will be fixed this week (v0.4.2), currently works with `master`.

In [ ]:
full_model = Model([
    gwpop.models.mass.two_component_primary_mass_ratio,
    gwpop.models.spin.iid_spin_magnitude_beta,
    gwpop.models.spin.independent_spin_orientation_gaussian_isotropic,
    gwpop.models.redshift.PowerLawRedshift()])

## Update sampling prior

We need to update the sampling prior to account for the new redshift evolution model.

Fortunately, we defined an interpolant for this earlier.

In [ ]:
for posterior in posteriors:
    posterior['prior'] *= redshift_prior(posterior['redshift'])

## Redshift dependent VT

Data from Maya

This again uses the `GridVT` with a 3D grid in primary mass, mass ratio, and redshift.

I apply a constant factor, this was found by brute force to make the rate look right.

In [ ]:
vt_file = '/home/colm.talbot/O2/population/data/O2_populations_maya/data/PDETS_GstLAL-PSDs.hdf5'
    
pdet_redshift = dd.io.load(vt_file)
pdet_vt_interp = RegularGridInterpolator(
    (pdet_redshift['ms'], pdet_redshift['ms'], pdet_redshift['zs']),
    pdet_redshift['int_pdet_L1'], bounds_error=False, fill_value=0)
m1s = np.linspace(5.95, 100, 200)
qs = np.linspace(0.01, 1, 100)
zs = np.linspace(1e-6, 1, 40)

m1_grid, q_grid, z_grid = np.meshgrid(m1s, qs, zs)
pdets = pdet_vt_interp(np.array([m1_grid * q_grid, m1_grid, z_grid]).T).T
pdet_dict = dict(
    mass_1=m1_grid, mass_ratio=q_grid, redshift=z_grid, vt=pdets)
for key in pdet_dict:
    pdet_dict[key] = np.asarray(pdet_dict[key])
z_grid_vt = gwpop.vt.GridVT(
    model=[gwpop.models.mass.two_component_primary_mass_ratio,
           gwpop.models.redshift.PowerLawRedshift()], data=pdet_dict)

def vt_eval_z_grid(parameters):
    return z_grid_vt(parameters) / 0.040109860595670524    

## Define the likelihood

Here we use the likelihood which also estimates the rate.

We actually lose our resolving power when we include the redshift evolution parameter.

We add an extra argument:
- `conversion_function`: this converts between the parameters we sample in and those needed by the model, e.g., for sampling in the mean and variance of the beta distribution

In [ ]:
full_likelihood = gwpop.hyperpe.RateLikelihood(
    posteriors=posteriors, hyper_prior=full_model, selection_function=vt_eval_z_grid,
    conversion_function=gwpop.conversions.convert_to_beta_parameters)

## Prior

In [ ]:
full_priors = PriorDict()

# rate
full_priors['rate'] = LogUniform(minimum=1e-20, maximum=1e20, latex_label='$R$')
# mass
full_priors['alpha'] = Uniform(minimum=-4, maximum=12, latex_label='$\\alpha$')
full_priors['beta'] = Uniform(minimum=-4, maximum=12, latex_label='$\\beta$')
full_priors['mmin'] = Uniform(minimum=5, maximum=10, latex_label='$m_{\\min}$')
full_priors['mmax'] = Uniform(minimum=20, maximum=60, latex_label='$m_{\\max}$')
full_priors['lam'] = Uniform(minimum=0, maximum=1, latex_label='$\\lambda_{m}$')
full_priors['mpp'] = Uniform(minimum=20, maximum=50, latex_label='$\\mu_{m}$')
full_priors['sigpp'] = Uniform(minimum=0, maximum=10, latex_label='$\\sigma_{m}$')
# spin magnitude
full_priors['amax'] = 1
full_priors['alpha_chi'] = Uniform(minimum=-4, maximum=12, latex_label='$\\alpha_{\\chi}$')
full_priors['beta_chi'] = Uniform(minimum=-4, maximum=12, latex_label='$\\beta_{\\chi}$')
# spin orientation
full_priors['xi_spin'] = Uniform(minimum=0, maximum=1, latex_label='$\\xi$')
full_priors['sigma_1'] = Uniform(minimum=0, maximum=4, latex_label='$\\sigma{1}$')
full_priors['sigma_2'] = Uniform(minimum=0, maximum=4, latex_label='$\\sigma{2}$')
# redshift evolution
full_priors['lamb'] = Uniform(minimum=-25, maximum=25, latex_label='$\\lambda_{z}$')

## Run sampler

This will take a while... use a GPU.

In [ ]:
full_result = bb.run_sampler(
    likelihood=full_likelihood, priors=full_priors, sampler='dynesty',
    nlive=1000, label='full')

_ = full_result.plot_corner(save=False)

In [1]:
d = dict(a=[1,11], b=[11])
list(d.values())[0]

[1, 11]

In [6]:
import glob
from configargparse import Namespace
files = glob.glob("/home/zoheyr.doctor/public_html/O3/O3aCatalog/data_release/all_posterior_samples/GW*[!comoving].h5")

In [24]:
#!/usr/bin/env python3
import glob
import json
import os
import re
from copy import deepcopy

import h5py
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy import units
from astropy.cosmology import Planck15
from bilby.core.utils import logger
from scipy.interpolate import interp1d
import tqdm


matplotlib.rcParams["text.usetex"] = False


def lalinference_prior(samples):
    """
    Evaluate lalinference prior in terms of
        - source frame primary mass
        - mass ratio
        - redshift

    p(m_1, q, z) = J p(m_1_det, m_2_det, dl) \propto dl^2 J
    J = dm_1_det / dm_1 dm_2_det / dq ddl / dz
    dm_1_det / dm_1 = (1 + z)
    dm_2_det / dq = m_1_det = m_1 (1 + z)
    ddl / dz = dl / (1 + z) + (1 + z) dH / E(z)
    J = m_1 (1 + z) ** 2 * (dl / (1 + z) + (1 + z) dH / E(z))
    """
    redshift = samples["redshift"]
    luminosity_distance = Planck15.luminosity_distance(redshift).to(units.Gpc).value
    return (
        luminosity_distance ** 2
        * samples["mass_1"]
        * (1 + redshift) ** 2
        * (
            luminosity_distance / (1 + redshift)
            + (1 + redshift)
            * Planck15.hubble_distance.to(units.Gpc).value
            / Planck15.efunc(redshift)
        )
    )


def replace_keys(posts, parameters_to_keep=[]):
    _mapping = dict(
        mass_1="m1_source",
        mass_2="m2_source",
        mass_ratio="q",
        a_1="a1",
        a_2="a2",
        cos_tilt_1="costilt1",
        cos_tilt_2="costilt2",
    )
    new_posts = dict()
    for name in posts:
        post = posts[name]
        new = pd.DataFrame()
        for key in _mapping:
            if _mapping[key] in post:
                new[key] = post[_mapping[key]]
            elif key in post:
                new[key] = post[key]
            else:
                new[key] = 0
        if "redshift" in post:
            new["redshift"] = post["redshift"]
            logger.info(f"Assuming LALInference redshift prior for {name}.")
            new["prior"] = lalinference_prior(new)
        else:
            logger.info(f"No redshift for {name}, setting prior to one.")
            new["prior"] = 1
        for key in parameters_to_keep:
            if key not in _mapping.keys():
                new[key] = post[key]
        new_posts[name] = new
    return new_posts


def load_gwtc_1_events(args):
    files = glob.glob(args.gwtc1_samples_regex)
    z_array = np.expm1(np.linspace(np.log(1), np.log(10 + 1), 1000))
    distance_array = Planck15.luminosity_distance(z_array).to(units.Mpc).value
    z_of_d = interp1d(distance_array, z_array)
    approximant = "Overall"
    posteriors = dict()
    meta_data = dict()
    for filename in files:
        posterior = pd.DataFrame()
        with h5py.File(filename, "r") as ff:
            try:
                data = np.array(ff[f"{approximant}_posterior"])
            except KeyError as e:
                logger.info(f"Failed to load {filename} with KeyError: {e}")
                continue
            posterior["redshift"] = z_of_d(data["luminosity_distance_Mpc"])
            for ii in [1, 2]:
                posterior[f"mass_{ii}"] = data[f"m{ii}_detector_frame_Msun"] / (
                    1 + posterior["redshift"]
                )
                posterior[f"a_{ii}"] = data[f"spin{ii}"]
                posterior[f"cos_tilt_{ii}"] = data[f"costilt{ii}"]
            posterior["mass_ratio"] = posterior["mass_2"] / posterior["mass_1"]
        posteriors[filename] = posterior
        meta_data[filename] = dict(label="GWTC-1", approximant=approximant)
        logger.info(f"Loaded {approximant} from {filename}.")
    return posteriors


def load_posterior_from_meta_file(filename):
    _mapping = dict(
        mass_1="mass_1_source",
        mass_2="mass_2_source",
        mass_ratio="mass_ratio",
        redshift="redshift",
        a_1="a_1",
        a_2="a_2",
        cos_tilt_1="cos_tilt_1",
        cos_tilt_2="cos_tilt_2",
        spin_1x="spin_1x",
        spin_1y="spin_1y",
        spin_1z="spin_1z",
        spin_2x="spin_2x",
        spin_2y="spin_2y",
        spin_2z="spin_2z",
    )
    labels = ["PrecessingSpinIMRHM", "PrecessingSpin"]
    approximants = [
        "NRSur7dq4",
        "IMRPhenomPv3HM",
        "IMRPhenomPv2",
    ]
    if not os.path.exists(filename):
        raise FileNotFoundError(f"{filename} does not exist")
    with h5py.File(filename, "r") as data:
        for label in labels:
            if label not in data.keys():
                continue
            _posterior = pd.DataFrame(data[label]["posterior_samples"][:])
            posterior = pd.DataFrame(
                {key: _posterior[_mapping[key]] for key in _mapping}
            )
            meta_data = dict(approximant=label.split(":")[-1], label=label)
            logger.info(f"Loaded {label} from {filename}.")
            return posterior, meta_data


def load_o3a_events(args):
    all_files = glob.glob(args.o3a_samples_regex)
    logger.info(f"Found {len(all_files)} O3 events in standard format.")
    posteriors, meta_data = _load_batch_of_meta_files(filenames=all_files)
    with open(os.path.join(args.run_dir, "data", "event_data.json"), "w") as ff:
        json.dump(meta_data, ff)
    return posteriors


def load_dat_files(args):
    """Load dat posteriors here (useful if some samples pre-processed)

    Note: only those param passed in the args will be returned

    Returns: dict(posterior_filename : Samples as pd.DataFrame)
    """
    all_files = glob.glob(args.dat_samples_regex)
    logger.info(f"Found {len(all_files)} ' ' delimited sample files.")
    posteriors = dict()
    for file in all_files:
        posterior = pd.read_csv(file, sep=" ", index_col=False)[args.parameters]
        posterior['m1_source'] = posterior['mass_1_source']
        posterior['m2_source'] = posterior['mass_2_source']
        posteriors.update({
                file: posterior
             })
        logger.info(f"Loaded samples from {file}.")

    return posteriors


def _load_batch_of_meta_files(filenames):
    keys = [
        "mass_1",
        "mass_ratio",
        "a_1",
        "a_2",
        "cos_tilt_1",
        "cos_tilt_2",
        "redshift",
        "spin_1x",
        "spin_1y",
        "spin_1z",
        "spin_2x",
        "spin_2y",
        "spin_2z",
    ]
    posteriors = dict()
    meta_data = dict()
    for posterior_file in filenames:
        try:
            new_posterior, data = load_posterior_from_meta_file(posterior_file)
        except TypeError as e:
            logger.info(f"Failed to load {posterior_file} with TypeError: {e}.")
            continue
        if all([key in new_posterior for key in keys]):
            meta_data[posterior_file] = data
            new_posterior = new_posterior[keys]
            if min(new_posterior["mass_ratio"]) >= 1:
                new_posterior["mass_ratio"] = 1 / new_posterior["mass_ratio"]
            posteriors[posterior_file] = new_posterior
        else:
            logger.info(f"Posterior has keys {new_posterior.keys()}.")
    return posteriors, meta_data


def load_all_events(args, funcs=None):
    if funcs is None:
        funcs = [load_gwtc_1_events, load_o3a_events]
    elif not isinstance(funcs, list):
        funcs = [funcs]
    posteriors = dict()
    logger.info("Loading posteriors...")
    for func in funcs:
        posteriors.update(func(args))
    logger.info("ADDING AGN SPIN")
#     print(posteriors)
    posteriors = add_agn_samples_to_posteriors(posteriors)
    posteriors = replace_keys(posteriors, args.parameters)
    logger.info(f"Loaded {len(posteriors)} posteriors.")
    return posteriors


def plot_summary(posteriors: list, events: list, args):
    posteriors = deepcopy(posteriors)
    plot_dir = os.path.join(args.run_dir, "data")
    plot_chi = True
    for posterior in posteriors:
        if all(
                [
                    key in posterior
                    for key in ["a_1", "a_2", "cos_tilt_1", "cos_tilt_2", "mass_ratio"]
                ]
        ):
            posterior["chi_eff"] = (
                                           posterior["a_1"] * posterior["cos_tilt_1"]
                                           + posterior["a_2"] * posterior[
                                               "cos_tilt_2"] * posterior["mass_ratio"]
                                   ) / (1 + posterior["mass_ratio"])
            posterior["chi_p"] = np.maximum(
                posterior["a_1"] * (1 - posterior["cos_tilt_1"] ** 2) ** 0.5,
                posterior["mass_ratio"]
                * (3 + 4 * posterior["mass_ratio"])
                / (4 + 3 * posterior["mass_ratio"])
                * posterior["a_2"]
                * (1 - posterior["cos_tilt_2"] ** 2) ** 0.5,
            )
        else:
            plot_chi = False
    plot_mass_2 = True
    for posterior in posteriors:
        try:
            posterior["mass_2"] = posterior["mass_1"] * posterior["mass_ratio"]
        except KeyError:
            plot_mass_2 = False
    for parameter in posteriors[0].keys():
        if parameter in ["chi_eff", "chi_p"] and not plot_chi:
            continue
        elif parameter == "mass_2" and not plot_mass_2:
            continue
        logger.info(f"Making box plot for {parameter}")
        file_name = os.path.join(plot_dir, parameter)
        data = [post[parameter] for post in posteriors]
        fig = plt.figure(figsize=(len(posteriors), 5))
        plt.violinplot(data)
        plt.ylabel(parameter.replace("_", " "))
        plt.xticks(np.arange(1, len(events) + 1), events, rotation=90)
        if parameter == "prior":
            plt.yscale("log")
        plt.tight_layout()
        plt.savefig(file_name + ".png")
        plt.close(fig)


def get_event_name(fname):
    return re.findall(r"(\w*\d{6}[a-z]*)", fname)[0]

def add_agn_samples_to_df(df):
    df['s1x'], df['s1y'], df['s1z'] = df['spin_1x'], df['spin_1y'], df['spin_1z']
    df['s2x'], df['s2y'], df['s2z'] = df['spin_2x'], df['spin_2y'], df['spin_2z']
    df['s1_dot_s2'] = (df['s1x'] * df['s2x']) + (df['s1y'] * df['s2y']) + (
            df['s1z'] * df['s2z'])
    df['s1_mag'] = np.sqrt(
        (df['s1x'] * df['s1x']) + (df['s1y'] * df['s1y']) + (df['s1z'] * df['s1z']))
    df['s2_mag'] = np.sqrt(
        (df['s2x'] * df['s2x']) + (df['s2y'] * df['s2y']) + (df['s2z'] * df['s2z']))
    df['cos_theta_12'] = df['s1_dot_s2'] / (df['s1_mag'] * df['s2_mag'])
    return df

def add_agn_samples_to_posteriors(posteriors):
    for f, d in tqdm.tqdm(posteriors.items(), desc='Add AGN samples to Posteriors'):
        event = get_event_name(f)
        try:
            d = add_agn_samples_to_df(d)
            posteriors.update({f: d})
        except Exception as e:
            print(f"Skipping {event}: {e}\n{d.columns.values.tolist()}")
    return posteriors


def gather_posteriors(args):
    if args.ignore is None:
        args.ignore = list()
    logger.info(f"Outdir is {args.run_dir}")
    if args.gwtc1 and not args.o3a:
        posteriors = load_all_events(args=args, funcs=[load_gwtc_1_events])
    elif args.o3a and not args.gwtc1:
        posteriors = load_all_events(args=args, funcs=[load_o3a_events])
    elif args.dat and not (args.gwtc1 or args.o3a):
        posteriors = load_all_events(args=args, funcs=[load_dat_files])
    elif all([args.gwtc1, args.o3a]):
        posteriors = load_all_events(args)
    else:
        logger.info("No events specified, loading all events.")
        posteriors = load_all_events(args)



    posts = list()
    events = list()
    with open(f"{args.run_dir}/data/{args.data_label}_posterior_files.txt", "w") as ff:
        for filename in posteriors.keys():
            ignore = False
            for label in args.ignore:
                if label in filename:
                    ignore = True
            if ignore:
                logger.info(f"Ignoring {filename}.")
                continue
            event = re.findall(r"(\w*\d{6}[a-z]*)", filename)[0]
            if event in events:
                continue
            posts.append(posteriors[filename])
            events.append(event)
            ff.write(f"{event}: {filename}\n")
    return posts, events




In [26]:
from configargparse import Namespace

args = Namespace(
    ignore=[],
    parameters=[ 'cos_theta_12'],
#     o3a_samples_regex="/home/zoheyr.doctor/public_html/O3/O3aCatalog/data_release/all_posterior_samples/GW*_comoving.h5",
    o3a_samples_regex="/home/shanika.galaudage/O3/population/o3a_pe_samples_release/S*.h5",
    dat=False, gwtc1=False, o3a=True,
    run_dir='test',
    data_label='test'
)
posts, events = gather_posteriors(args=args)

16:43 bilby INFO    : Outdir is test
16:43 bilby INFO    : Loading posteriors...
16:43 bilby INFO    : Found 39 O3 events in standard format.
16:43 bilby INFO    : Loaded PrecessingSpinIMRHM from /home/shanika.galaudage/O3/population/o3a_pe_samples_release/S190408an.h5.
16:43 bilby INFO    : Loaded PrecessingSpinIMRHM from /home/shanika.galaudage/O3/population/o3a_pe_samples_release/S190412m.h5.
16:43 bilby INFO    : Loaded PrecessingSpinIMRHM from /home/shanika.galaudage/O3/population/o3a_pe_samples_release/S190413ac.h5.
16:43 bilby INFO    : Loaded PrecessingSpinIMRHM from /home/shanika.galaudage/O3/population/o3a_pe_samples_release/S190413i.h5.
16:43 bilby INFO    : Loaded PrecessingSpinIMRHM from /home/shanika.galaudage/O3/population/o3a_pe_samples_release/S190421ar.h5.
16:43 bilby INFO    : Loaded PrecessingSpinIMRHM from /home/shanika.galaudage/O3/population/o3a_pe_samples_release/S190424ao.h5.
16:43 bilby INFO    : Failed to load /home/shanika.galaudage/O3/population/o3a_pe_samp

{'/home/shanika.galaudage/O3/population/o3a_pe_samples_release/S190408an.h5':           mass_1  mass_ratio       a_1       a_2  cos_tilt_1  cos_tilt_2  \
0      24.777434    0.877990  0.553222  0.310523   -0.382521    0.945086   
1      21.656837    0.857207  0.777600  0.520862   -0.372569   -0.029071   
2      21.325763    0.983181  0.473670  0.634391    0.324260   -0.274648   
3      22.898892    0.829648  0.758463  0.402229    0.436187   -0.671067   
4      23.285279    0.903770  0.817927  0.388688   -0.358257    0.683172   
...          ...         ...       ...       ...         ...         ...   
37361  30.301706    0.512860  0.108830  0.716262    0.583490    0.021557   
37362  24.709623    0.628772  0.491680  0.156617   -0.843884   -0.635701   
37363  20.364448    0.983181  0.473670  0.634391    0.324260   -0.274648   
37364  24.043337    0.859771  0.290820  0.055537   -0.263955   -0.804390   
37365  29.225161    0.660533  0.249627  0.199338   -0.120056    0.452388   

       re

Add AGN samples to Posteriors: 100%|██████████| 38/38 [00:00<00:00, 47.92it/s]
16:43 bilby INFO    : Assuming LALInference redshift prior for /home/shanika.galaudage/O3/population/o3a_pe_samples_release/S190408an.h5.
16:43 bilby INFO    : Assuming LALInference redshift prior for /home/shanika.galaudage/O3/population/o3a_pe_samples_release/S190412m.h5.
16:43 bilby INFO    : Assuming LALInference redshift prior for /home/shanika.galaudage/O3/population/o3a_pe_samples_release/S190413ac.h5.
16:43 bilby INFO    : Assuming LALInference redshift prior for /home/shanika.galaudage/O3/population/o3a_pe_samples_release/S190413i.h5.
16:43 bilby INFO    : Assuming LALInference redshift prior for /home/shanika.galaudage/O3/population/o3a_pe_samples_release/S190421ar.h5.
16:43 bilby INFO    : Assuming LALInference redshift prior for /home/shanika.galaudage/O3/population/o3a_pe_samples_release/S190424ao.h5.
16:43 bilby INFO    : Assuming LALInference redshift prior for /home/shanika.galaudage/O3/popul

In [27]:
posts

[          mass_1  mass_2  mass_ratio       a_1       a_2  cos_tilt_1  \
 0      24.777434       0    0.877990  0.553222  0.310523   -0.382521   
 1      21.656837       0    0.857207  0.777600  0.520862   -0.372569   
 2      21.325763       0    0.983181  0.473670  0.634391    0.324260   
 3      22.898892       0    0.829648  0.758463  0.402229    0.436187   
 4      23.285279       0    0.903770  0.817927  0.388688   -0.358257   
 ...          ...     ...         ...       ...       ...         ...   
 37361  30.301706       0    0.512860  0.108830  0.716262    0.583490   
 37362  24.709623       0    0.628772  0.491680  0.156617   -0.843884   
 37363  20.364448       0    0.983181  0.473670  0.634391    0.324260   
 37364  24.043337       0    0.859771  0.290820  0.055537   -0.263955   
 37365  29.225161       0    0.660533  0.249627  0.199338   -0.120056   
 
        cos_tilt_2  redshift        prior  cos_theta_12  
 0        0.945086  0.210624   237.634648     -0.059663  
 1    

In [20]:
f = "/home/zoheyr.doctor/public_html/O3/O3aCatalog/data_release/all_posterior_samples/GW190930_133541_comoving.h5"